In [1]:
import pandas as pd
from requests_html import HTMLSession

In [2]:
def get_links(url):
    session = HTMLSession()
    r = session.get(url)
    links = []
    for article_url in r.html.find('a.headline'):
        if 'href' in article_url.attrs:
            links.append(article_url.attrs['href'])
    return links

In [3]:
links = get_links('https://www.birminghammail.co.uk/all-about/crime')
links[:5]

['https://www.birminghammail.co.uk/news/midlands-news/man-cleared-raping-student-city-18838321',
 'https://www.birminghammail.co.uk/news/midlands-news/watch-shocking-120mph-police-chase-18855257',
 'https://www.birminghammail.co.uk/news/midlands-news/violent-moseley-thug-kicked-woman-18837561',
 'https://www.birminghammail.co.uk/news/midlands-news/remorseless-killer-who-stabbed-kingstanding-18825294',
 'https://www.birminghammail.co.uk/news/midlands-news/schoolboys-15-arrested-after-swords-18852777']

In [4]:
def get_articles(links):
    link = []
    title = []
    description = []
    author = []
    date_published = []
    date_updated = []
    article = []
    for url in links:
        session = HTMLSession()
        r = session.get(url)
        link.append(url)
        try:
            title.append(r.html.find('h1.section-theme-background-indicator.publication-font', first=True).text)
        except: title.append(None)
            
        try:
            description.append(r.html.find('p.sub-title', first=True).text)
        except: description.append(None)
            
        try:
            author.append(r.html.find('a.publication-theme', first=True).text)
        except: author.append(None)
            
        try:
            date_published_element = r.html.find('time.date-published', first=True)
            if 'datetime' in date_published_element.attrs:
                date_published.append(date_published_element.attrs['datetime'])
        except: date_published.append(None)
            
        try:
            date_updated_element = r.html.find('time.date-updated', first=True)
            if 'datetime' in date_updated_element.attrs:
                date_updated.append(date_updated_element.attrs['datetime'])
        except: date_updated.append(None)
            
        try:
            article_paragraphs = []
            for article_paragraph in r.html.find('div.article-body > p'):
                article_paragraphs.append(article_paragraph.text)
            article.append(article_paragraphs)
        except: article.append(None)

    articles_dict = {'link': link, 'title': title, 'description': description, 'article': article,
                     'author': author, 'date_published': date_published, 'date_updated': date_updated}
    return articles_dict 

In [5]:
articles_dict = get_articles(links)

In [6]:
articles_dict.keys()

dict_keys(['link', 'title', 'description', 'article', 'author', 'date_published', 'date_updated'])

In [7]:
#print(articles_dict['article'][1])
len(articles_dict['article'])

24

In [8]:
pd.DataFrame(articles_dict).head()

,link,title,description,article,author,date_published,date_updated
0,https://www.birminghammail.co.uk/news/midlands...,Man cleared of raping student at Birmingham ci...,Jamie Seaton met a student at a city centre pu...,[A man has been cleared of raping a student he...,Ross McCarthy,2020-08-31T14:00:00Z,2020-08-31T14:50:56Z
1,https://www.birminghammail.co.uk/news/midlands...,Watch shocking 120mph police chase which start...,A motorist led police on a 120mph pursuit over...,[This is the shocking footage of a 120mph poli...,Ben Perrin,2020-08-31T12:26:53Z,None
2,https://www.birminghammail.co.uk/news/midlands...,Violent Moseley thug kicked woman in head in d...,A judge admits taking 'huge risk' by not sendi...,[A violent thug kicked a woman in the head whi...,Carl Jackson,2020-08-31T04:30:00Z,2020-08-31T06:52:12Z
3,https://www.birminghammail.co.uk/news/midlands...,Remorseless killer who stabbed Kingstanding lo...,Peter Hastings knifed Jean Bellis 29 times in ...,[A remorseless killer was caught after he unwi...,Ben Perrin,2020-08-31T04:30:00Z,2020-08-31T06:52:46Z
4,https://www.birminghammail.co.uk/news/midlands...,"Schoolboys, 15, arrested after swords and mach...",Three teenagers were taken into custody for qu...,[Schoolboys have been arrested after a broad d...,Stephanie Balloo,2020-08-30T15:31:19Z,2020-08-30T15:32:37Z
